# py2neo 使用教程
[简书](https://www.jianshu.com/p/febe8a248582)

In [29]:
from py2neo import Graph,Node,Relationship
graph = Graph("bolt://localhost:7687",auth=("neo4j","password"))

## 链接

In [30]:
# 创建节点Node和关系Relationship
a = Node("Person", name="Alice")
b = Node("Person", name="Bob")
ab = Relationship(a, "KNOWS", b)
# graph.create(ab)

## 数据对象 Object
### Node

In [31]:
print(type(a))
node = a
print(node['name'], node)

<class 'py2neo.data.Node'>
Alice (:Person {name: 'Alice'})


In [32]:
node['age'] = '22'
print(node)

(:Person {age: '22', name: 'Alice'})


In [33]:
#将node的所有property以dictionary的形式返回
dict(node)

{'name': 'Alice', 'age': '22'}

In [34]:
del node['age']
node

Node('Person', name='Alice')

In [35]:
#返回所以和这个节点有关的label
labels = node.labels
labels, type(labels)

(:Person, py2neo.cypher.encoding.LabelSetView)

In [22]:
#删除某个label
node.labels.remove('Person')
node

TypeError: 'LabelSetView' object is not callable

In [36]:
help(node.labels)

Help on LabelSetView in module py2neo.cypher.encoding object:

class LabelSetView(py2neo.collections.SetView)
 |  LabelSetView(elements=(), selected=(), **kwargs)
 |  
 |  A set is a finite, iterable container.
 |  
 |  This class provides concrete generic implementations of all
 |  methods except for __contains__, __iter__ and __len__.
 |  
 |  To override the comparisons (presumably for speed, as the
 |  semantics are fixed), redefine __le__ and __ge__,
 |  then the other operations will automatically follow suit.
 |  
 |  Method resolution order:
 |      LabelSetView
 |      py2neo.collections.SetView
 |      collections.abc.Set
 |      collections.abc.Collection
 |      collections.abc.Sized
 |      collections.abc.Iterable
 |      collections.abc.Container
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __getattr__(self, element)
 |  
 |  __init__(self, elements=(), selected=(), **kwargs)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 

### Relationship

In [41]:
#创建Relationship
# ab = Relationship(a, "KNOWS", b)
# Relationship(*start_node*, *type*, *end_node*, ***properties*)
relationship = ab
#返回Relationship的property
print(relationship['KNOWS'])
#删除某个property
# del relationship[key]
#将relationship的所有property以dictionary的形式返回
dict(relationship)

None


{}

In [47]:
type(ab), ab

(py2neo.data.KNOWS,
 KNOWS(Node('Person', name='Alice'), Node('Person', name='Bob')))

## 查询 Query

或者使用 py2neo.matching包里面的NodeMatcher(graph) 函数构建个matcher再查询（其实上面的方法就调用的这个东西), 首先创建Matcher来执行查询，它会返回一个Match类型的数据，可以继续使用where(),first(),order_by等操作，可以用list强制转换

In [69]:
from py2neo import NodeMatcher

In [70]:
# 对node的查询
graph.nodes.match('Country').all()

[Node('Country', name='China')]

In [57]:
graph = Graph()
matcher = NodeMatcher(graph)
print(matcher.match("Person", name="Alice").first())
print(matcher.match("Person", name="Alice").all())
#结果 (_224:Person {born:1964,name:"Keanu Reeves"})

(_0:Person {name: 'Alice'})
[Node('Person', name='Alice'), Node('Person', name='Alice')]


In [58]:
# where里面使用_指向当前node
list(matcher.match("Person").where("_.name =~ 'A.*'"))

[Node('Person', name='Alice'), Node('Person', name='Alice')]

In [66]:
graph.match(nodes=None, r_type=None, limit=None).all() # 找到所有的relationships

[KNOWS(Node('Person', name='Alice'), Node('Person', name='Bob')),
 KNOWS(Node('Person', name='Alice'), Node('Person', name='Bob')),
 HoldIn(Node('WOG', num=24), Node('Country', name='China'))]

[Node('Country', name='China')]

## 更新

In [62]:
from py2neo import Graph, NodeMatcher, Subgraph

In [73]:
tx = graph.begin()
matcher = NodeMatcher(graph)
nodes = matcher.match("Country")
print(type(nodes))
# 将返回的“Match”类转成list
new_nodes = list(nodes)
print(new_nodes)

<class 'py2neo.matching.NodeMatch'>
[Node('Country', name='China')]


In [75]:
## 添加你要修改的东西
# for node in new_nodes:
#     node['tag'] = node['tag'].split(',')
# 里面是Node的list可以作为Subgraph的参数

sub = Subgraph(nodes=new_nodes)   # 由 Node List 更新
# 调用push更新
tx.push(sub)
tx.commit()

<Bookmark 'FB:kcwQ34ZR8trdSyaHqO4NndsVTwuQ'>

In [81]:
graph.nodes.match().all()

[Node('Person', name='Alice'),
 Node('Person', name='Bob'),
 Node('Person', name='Alice'),
 Node('Person', name='Bob'),
 Node('WOG', num=24),
 Node('Country', name='China')]

## 批处理
对于大量的插入一般是很费时的，首先我们可以使用事务，加快一定速度，
而插入的方法一样重要，我们很多时候是遍历一个文件然后生成图，
例子中我们生成每个Node后，先把他们放入一个List中，再变为Subgraph实例,然后再create(),耗时比一条条插入至少快10倍以上


In [ ]:
tx = graph.begin()
nodes=[]
for line in lineLists:
    oneNode = Node()
    ........
    #这里的循环，一般是把文件的数据存入node中
    nodes.append(oneNode)
nodes=neo.Subgraph(nodes)
tx.create(nodes)
tx.commit()

前面我们说了关系的创建，如果在node存进去后，再通过py2neo层面的“查找node,create关系”
这样的效率是很低的，时间主要花在通过"reference Id"去一个个去查找对应的node,
然后再和这个node建立关系，这里强烈推荐使用原生语句，效率不是一般的高


In [ ]:
#假定我们已经把两种node存进去了，label分别是Post和User,现在需要在他们间建立某关系
graph.run("MATCH (p:Post),(u:User) \
                WHERE p.OwnerUserId = u.Id \
                CREATE (u)-[:Own]->(p)")